# 応用編テキスト：モデルのチューニングと評価

### 1. モデルのハイパーパラメータチューニング
ハイパーパラメータとは、モデルの学習プロセスや構造を制御する設定項目です。主なハイパーパラメータは以下の通りです：

- 学習率: モデルがどれだけ早く収束するかを決定。
- バッチサイズ: 一度に学習するデータの量。
- エポック数: 全データセットを何回学習するか。
- ニューロン数・層の数: ネットワークの表現力を決定。
- ハイパーパラメータの調整は、性能に大きく影響します。小さな変更でも結果が変わるので、調整と評価を繰り返すことが重要です。

### 2. 過学習とその対策
過学習とは、モデルが訓練データに過剰に適合してしまい、未知のデータに対して性能が低下する現象です。

主な対策:

- データ拡張: データをランダムに変形し、見かけ上のデータ量を増やす。
- ドロップアウト: ニューラルネットワークの一部のノードをランダムに無効化して学習。
- 正則化: モデルの重みにペナルティを課す（L2正則化など）。

### 3. モデル評価指標
分類タスクの評価には以下の指標が役立ちます：

- 正解率（Accuracy）: 正しく分類された割合。
- 混同行列（Confusion Matrix）: 各クラスごとの予測結果の分布。
- F1スコア: 再現率と適合率の調和平均。

評価のために以下のツールを利用できます：

In [1]:
from sklearn.metrics import classification_report, confusion_matrix


### 例題1: ハイパーパラメータのチューニング
以下の例では、学習率とバッチサイズを変更してMNISTモデルの性能を確認します。

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# データ前処理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# モデル定義
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# ハイパーパラメータの設定
learning_rate = 0.01  # 学習率
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# トレーニングループ
for epoch in range(5):
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/5], Loss: {loss.item():.4f}')

# 評価
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy with learning rate {learning_rate}: {100 * correct / total:.2f}%')


Epoch [1/5], Loss: 0.3813
Epoch [2/5], Loss: 0.2811
Epoch [3/5], Loss: 0.1398
Epoch [4/5], Loss: 0.2539
Epoch [5/5], Loss: 0.0943
Accuracy with learning rate 0.01: 95.20%


### 例題2: 過学習防止（ドロップアウトの利用）
ドロップアウトを追加して、過学習を防止します。

In [3]:
class MLPWithDropout(nn.Module):
    def __init__(self):
        super(MLPWithDropout, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.dropout1 = nn.Dropout(0.5)  # ドロップアウト追加
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)  # ドロップアウト追加
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# ドロップアウトを使ったモデルを試してください


### 追加課題
学習率やバッチサイズを変えて、どの設定が最適か確認してください。

ドロップアウトの割合（0.2 や 0.8）を変更してみましょう。

混同行列を表示し、どの数字を誤認識しているか確認してください。